# Money Manager Demo (Mock Data)

This demo uses **fake bank data** so you can test the Money Manager without needing a real Stripe account!

Perfect for:
- Hackathon projects
- Testing without SSN/activation
- Quick demos

The fake data includes:
- 2 bank accounts (Checking: $2,450.75, Savings: $8,500.00)
- 17 realistic transactions (groceries, eating out, bills, income, etc.)
- Full Money Manager functionality

## Step 0: Install Dependencies

In [ ]:
# Install required packages
%pip install -q langchain langchain-openai langgraph pydantic python-dotenv
print("✅ Dependencies installed!")

## Step 1: Load API Keys

You only need your OpenRouter API key for the LLM - no Stripe needed!

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../secrets.env")
load_dotenv("../variables.env")

# Check for OpenRouter API key
openrouter_key = os.getenv("OPENROUTER_API_KEY")

if not openrouter_key:
    print("❌ OPENROUTER_API_KEY not found")
    print("   Add it to backend/agent/secrets.env")
else:
    print("✅ OpenRouter API key loaded!")
    print(f"   Key: {openrouter_key[:20]}...")

## Step 2: Initialize Mock Banking System

This creates fake bank accounts and transactions for testing.

In [ ]:
from stripe_integration.mock_client import MockStripeFinancialClient

# Create mock client with fake data
mock_client = MockStripeFinancialClient()

# Show what accounts we have
print("✅ Mock banking system ready!\n")
print("Your test accounts:")

accounts = await mock_client.get_accounts()
total = 0
for acc in accounts:
    print(f"  {acc.name}: ${acc.balance:,.2f}")
    total += acc.balance

print(f"\n  Total: ${total:,.2f}")
print("\n💡 This is all fake data - perfect for testing!")

## Step 3: Check Your Transactions

Let's see what fake transactions we have!

In [ ]:
# Get transactions for checking account
checking_account = accounts[0]
transactions = await mock_client.get_transactions(checking_account.id, days=30)

print(f"Recent transactions for {checking_account.name}:\n")

# Group by category
by_category = {}
for txn in transactions:
    if txn.category not in by_category:
        by_category[txn.category] = []
    by_category[txn.category].append(txn)

# Show by category
for category, txns in sorted(by_category.items()):
    total = sum(txn.amount for txn in txns)
    print(f"\n{category}: ${abs(total):,.2f}")
    for txn in txns[:3]:  # Show first 3
        sign = "+" if txn.amount > 0 else ""
        print(f"  {txn.merchant_name}: {sign}${txn.amount:,.2f}")
    if len(txns) > 3:
        print(f"  ... and {len(txns) - 3} more")

## Step 4: Initialize the Money Manager Agent

Now let's set up the AI agent that will help manage your money!

In [ ]:
# We need to modify the banking tools to use our mock client
# Let's create a simple wrapper

from financial_agent.shared_state import FinancialState, BankAccount as StateBankAccount, Transaction as StateTransaction

# Create initial state
state = FinancialState(
    user_id="demo_user",
    stripe_session_id="mock_session_123"  # Fake session ID
)

# Load accounts into state
state.accounts = [
    StateBankAccount(
        id=acc.id,
        name=acc.name,
        type=acc.type,
        balance=acc.balance,
        currency=acc.currency,
        institution_name=acc.institution_name,
        last_four=acc.last_four
    )
    for acc in accounts
]

print("✅ Money Manager initialized!")
print(f"\nManaging {len(state.accounts)} accounts with ${sum(acc.balance for acc in state.accounts):,.2f} total")

## Step 5: Simple Banking Operations

Let's test some basic operations without the full AI agent first.

In [ ]:
# Test 1: Check balances
print("💰 Your Money:\n")
for acc in state.accounts:
    print(f"  {acc.name}")
    print(f"    Balance: ${acc.balance:,.2f}")
    print(f"    Type: {acc.type.title()}")
    print(f"    Bank: {acc.institution_name}")
    print()

In [ ]:
# Test 2: Transfer money
print("Testing transfer: Moving $100 from Checking to Savings\n")

checking = state.accounts[0]
savings = state.accounts[1]

print(f"Before:")
print(f"  Checking: ${checking.balance:,.2f}")
print(f"  Savings: ${savings.balance:,.2f}")

# Do the transfer
success = await mock_client.transfer_funds(
    checking.id,
    savings.id,
    100.00,
    "Test transfer"
)

if success:
    # Reload accounts to get updated balances
    updated_accounts = await mock_client.get_accounts()
    checking = updated_accounts[0]
    savings = updated_accounts[1]

    print(f"\n✅ Transfer successful!\n")
    print(f"After:")
    print(f"  Checking: ${checking.balance:,.2f}")
    print(f"  Savings: ${savings.balance:,.2f}")
else:
    print("\n❌ Transfer failed")

## Step 6: Talk to the Money Manager (Simple Version)

Since the full LangGraph agent needs modification to use mock data, here's a simple demo of what it would say:

In [ ]:
def simple_money_manager_response(question: str) -> str:
    """Simple rule-based responses to demonstrate the Money Manager"""

    question_lower = question.lower()

    # Reload current accounts
    accounts = [acc async for acc in mock_client.get_accounts()]

    if "how much" in question_lower or "balance" in question_lower:
        total = sum(acc.balance for acc in accounts)
        response = "Let me check all your accounts for you!\n\n"
        response += "Your Money:\n"
        for acc in accounts:
            response += f"\n💰 {acc.name}: ${acc.balance:,.2f}\n"
            response += f"   This is your {acc.type} account for "
            if acc.type == "checking":
                response += "everyday spending and bills.\n"
            else:
                response += "saving money and emergencies.\n"

        response += f"\nTotal across all accounts: ${total:,.2f}\n\n"

        if accounts[0].balance > 1000:
            response += "✓ Your checking account looks healthy! That should cover your bills this month."
        else:
            response += "⚠️ Your checking is getting low. Want to check your budget?"

        return response

    elif "spent" in question_lower or "spending" in question_lower:
        # This would use real transaction analysis
        return """Let me show you what you've been spending on recently!

What You've Spent This Month:

🛒 Groceries - $202.00
   • Whole Foods ($87.50)
   • Trader Joe's ($69.30)
   • Safeway ($45.20)

🍔 Eating Out - $91.75
   • Local Restaurant ($45.00)
   • Pizza Place ($28.50)
   • Chipotle ($12.50)
   • Coffee ($5.75)

🚗 Transportation - $83.50
   • Gas ($65.00)
   • Uber ($18.50)

📦 Shopping - $135.29
   • Amazon ($89.99)
   • Target ($45.30)

💡 Your biggest expense this month was groceries - that's totally normal!
You spent about $91.75 eating out, which is about $3 per day."""

    elif "move" in question_lower or "transfer" in question_lower:
        return """I can help you move money between accounts!

Before we do that, let me show you what it would look like:

Current balances:
  • Checking: $2,350.75
  • Savings: $8,600.00

How much would you like to move, and which direction?
(For example: "Move $200 from checking to savings")

I'll always show you the before and after amounts and ask for confirmation before moving any money!"""

    else:
        return """I'm your friendly Money Manager! I can help you:

• Check your account balances ("How much money do I have?")
• See what you've been spending ("What did I spend this week?")
• Move money between accounts ("Move $100 to savings")
• Get a summary of everything ("Give me a summary")

What would you like to know?"""

# Test it out
question = "How much money do I have?"
print(f"You: {question}\n")
print("Money Manager:")
print(await simple_money_manager_response(question))

In [ ]:
# Try another question
question = "What did I spend on this month?"
print(f"You: {question}\n")
print("Money Manager:")
print(await simple_money_manager_response(question))

## Summary

This demo shows:

✅ **Mock bank accounts** with realistic balances  
✅ **Fake transactions** across different categories  
✅ **Transfer functionality** that updates balances  
✅ **Beginner-friendly explanations** in simple language  

**Perfect for hackathons!** No SSN, no real bank connection needed.

### Next Steps:

1. **For your stock analysis**: Use the existing `docgen_agent` - it already works!
2. **For the full Money Manager with AI**: Modify the banking tools to use `MockStripeFinancialClient`
3. **For Budget Helper**: Build it next using this same mock data approach

**Want to integrate everything?** The mock client has the same interface as the real Stripe client, so you can swap them easily!